This notebook will compare model architecture, hyperparameters, inputs, and training/validation error

In [1]:
import xarray as xr
import numpy as np
from src.crossval import run_crossval

In [2]:
input_dir = "/glade/work/milesep/convective_outlook_ml"
target_dir = "data/processed_data"
stats_dir = "data/processed_data"

In [5]:
# compare all models, save training and validation curves along with corresponding model name, data level, model/training hyperparameters
model_names = ["linear_regression"]
levels = ["small"]
# Don't do linear regression with full dataset--too many parameters
for name, level in zip(model_names, levels):
    print(name, level)
    inputs = xr.open_zarr(f"{input_dir}/train_inputs_{level}.zarr")
    targets = xr.open_dataset(f"{target_dir}/train_targets.nc")
    stats = xr.open_dataset(f"{stats_dir}/daily_input_stats_{level}.nc")
    scores = run_crossval(inputs, targets, name, epochs = 40, level = level, restart = False)
    print(f"{name}: {np.mean(scores):.3f} ± {np.std(scores):.3f}")

linear_regression small

Fold 0:
Resuming training from epoch 2/40
  Epoch 3/40 — Train Loss: 84.1825, Val Loss: 54.0264
  Epoch 4/40 — Train Loss: 82.6561, Val Loss: 22.5831
  Epoch 5/40 — Train Loss: 47.8445, Val Loss: 28.3943
  Epoch 6/40 — Train Loss: 29.7669, Val Loss: 25.2099
  Epoch 7/40 — Train Loss: 26.4385, Val Loss: 8.6109
  Epoch 8/40 — Train Loss: 21.0711, Val Loss: 6.6179
  Epoch 9/40 — Train Loss: 12.7065, Val Loss: 8.0260
  Epoch 10/40 — Train Loss: 8.9884, Val Loss: 6.5138
  Epoch 11/40 — Train Loss: 9.1586, Val Loss: 5.1406
  Epoch 12/40 — Train Loss: 6.3795, Val Loss: 4.2097
  Epoch 13/40 — Train Loss: 5.4790, Val Loss: 3.1361
  Epoch 14/40 — Train Loss: 3.8228, Val Loss: 3.0012
  Epoch 15/40 — Train Loss: 3.0694, Val Loss: 3.4821
  Epoch 16/40 — Train Loss: 2.4696, Val Loss: 2.5187
  Epoch 17/40 — Train Loss: 2.2178, Val Loss: 2.6386
  Epoch 18/40 — Train Loss: 1.6110, Val Loss: 3.0864
  Epoch 19/40 — Train Loss: 1.5925, Val Loss: 2.4180
  Epoch 20/40 — Train Loss: 

KeyboardInterrupt: 